In [1]:
pip install pandas mysql-connector-python

^C
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import mysql.connector

config = {
    'user': 'root',
    'password': 'Deusebom7$',
    'host': 'localhost',  
    'database': 'sakila',
}

In [5]:
conn = mysql.connector.connect(**config)

In [14]:
rentals_may = """select customer_id, count(*)
                    from rentals_may
                        group by customer_id 
                            order by count(*) desc;"""

In [15]:
rentals_june = """select customer_id, count(*)
                    from rentals_june
                        group by customer_id
                            order by count(*) desc;"""

In [16]:
df1 = pd.read_sql_query(rentals_may, conn)
df1

C:\Users\Ádamo\AppData\Local\Temp\ipykernel_15196\984176238.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df1 = pd.read_sql_query(rentals_may, conn)


,customer_id,count(*)
0,197,8
1,506,7
2,109,7
3,269,6
4,239,6
...,...,...
515,431,1
516,351,1
517,10,1
518,136,1


In [17]:
df2 = pd.read_sql_query(rentals_june, conn)
df2

C:\Users\Ádamo\AppData\Local\Temp\ipykernel_15196\1692617289.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df2 = pd.read_sql_query(rentals_june, conn)


,customer_id,count(*)
0,31,11
1,454,10
2,329,9
3,295,9
4,561,9
...,...,...
585,425,1
586,412,1
587,335,1
588,22,1


In [18]:
def compare_rentals_may_june(df1, df2):
    merged_df = pd.merge(df1, df2, on='customer_id', how='outer', suffixes=('_may', '_june'))
    
    merged_df.fillna(0, inplace=True)
    
    merged_df['comparison'] = merged_df['count(*)_june'] - merged_df['count(*)_may']
    
    merged_df['comparison_result'] = merged_df['comparison'].apply(lambda x: 'More' if x > 0 else 'Less' if x < 0 else 'Equal')
    
    return merged_df

In [19]:
results = compare_rentals_may_june(df1, df2)
results

,customer_id,count(*)_may,count(*)_june,comparison,comparison_result
0,197,8.0,8.0,0.0,Equal
1,506,7.0,5.0,-2.0,Less
2,109,7.0,5.0,-2.0,Less
3,269,6.0,3.0,-3.0,Less
4,239,6.0,5.0,-1.0,Less
...,...,...,...,...,...
593,63,0.0,1.0,1.0,More
594,598,0.0,1.0,1.0,More
595,487,0.0,1.0,1.0,More
596,555,0.0,1.0,1.0,More
